### Training model

- Decide which feature to use(just try first)
- Split training data to training data set and validation data set
- Select serval models to train
- Generate submission file (change column name 'date' back to 'day' before submission)


### Feature explanation

- year, year of data
- month, month of year 
- day, day of month
- hour, hour of day from 0 ~ 23
- weekday, week of day from 0 to 6, 0 is Mon, 1 is Tue, 2 is Wed etc.
- grid_id, grid id defined in grid info
- temperture, temperture at given time
- visibility, 0 visibility is bad, 1 visiblity is good
- wind, 0 means no wind or small wind, 1 means wind is strong
- rainy, 0 means no rain, 1 means is rainy
- holiday, 0 means working day, 1 means holiday
- car_number, number of cars in that grid at given time

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import timeit
%matplotlib inline

training_features_filepath = 'data/training.csv'
filtered_training_features_filepath = 'data/filtered_train.csv'
test_features_filepath = 'data/test.csv'
submission_filepath = 'data/submit_samples.csv'
new_submission_filepath = 'data/submission.csv'

drop_columns = ['car_number','year']

# loading data
train = pd.read_csv(training_features_filepath)
train.head()

,year,month,day,hour,weekday,grid_id,temperture,visibility,wind,rainy,holiday,car_number
0,2017,1,2,0,0,2,11,1,0,0,1,1
1,2017,1,2,0,0,4,11,1,0,0,1,5
2,2017,1,2,0,0,5,11,1,0,0,1,3
3,2017,1,2,0,0,6,11,1,0,0,1,3
4,2017,1,2,0,0,7,11,1,0,0,1,1


In [3]:
test = pd.read_csv(test_features_filepath)
test.head()

,year,month,day,hour,weekday,grid_id,temperture,visibility,wind,rainy,holiday,car_number
0,2017,3,13,9,0,1,9,1,1,1,0,0
1,2017,3,13,9,0,2,9,1,1,1,0,0
2,2017,3,13,9,0,3,9,1,1,1,0,0
3,2017,3,13,9,0,4,9,1,1,1,0,0
4,2017,3,13,9,0,5,9,1,1,1,0,0


In [4]:
def remove_chinese_new_year_data(df):
    df = df[(df.month != 1) | (df.day != 27)]
    df = df[(df.month != 1) | (df.day != 28)]
    df = df[(df.month != 1) | (df.day != 29)]
    df = df[(df.month != 1) | (df.day != 30)]
    df = df[(df.month != 1) | (df.day != 31)]
    df = df[(df.month != 2) | (df.day != 1)]
    df = df[(df.month != 2) | (df.day != 2)]
    return df

def remove_other_hours_data(df):
    df = df[(df.hour != 0)]
    df = df[(df.hour != 1)]
    df = df[(df.hour != 2)]
    df = df[(df.hour != 3)]
    df = df[(df.hour != 4)]
    df = df[(df.hour != 5)]
    df = df[(df.hour != 6)]
    df = df[(df.hour != 7)]
    df = df[(df.hour != 8)]
    df = df[(df.hour != 23)]
    return df

def retrieve_index_by_month_day_hour(df, month, day, hour):
    month = df[df['month'] == month]
    day = month[month['day'] == day]
    hour = day[day['hour'] == hour]
    return hour.index[0]

def generate_submission_file(pred):
    sample_df = pd.read_csv(submission_filepath)
    sample_df['car_number'] = pred
    sample_df['car_number'] = sample_df.car_number.round()
    sample_df['car_number'] = sample_df['car_number'].astype(int)
    sample_df.columns = ['grid_id','day','hour','car_number']
    sample_df.to_csv(new_submission_filepath, index=False)

def split_train_val_data(index):
    # split train and val by index
    train_ = train.iloc[:index]
    val_ = train.iloc[index:-1]
    # extract x and y
    train_x = train_.drop(columns=drop_columns)
    train_y = train_['car_number']
    val_x = val_.drop(columns=drop_columns)
    val_y = val_['car_number']
    return train_x, train_y, val_x, val_y

### Model selecting

- XGBoost

### parameter tuning


- 'subsample' : 0.70 -> score : 11.347555193983732
- 'subsample' : 0.75 -> score : 11.32022463662512
- 'subsample' : 0.80 -> score : 11.362855348692083
- 'subsample' : 0.85 -> score : 11.357121493573116
- 'subsample' : 0.90 -> score : 11.342456547514821


- 'colsample_bytree' : 0.70 -> score : 11.32022463662512
- 'colsample_bytree' : 0.80 -> score : 11.289090541110422
- 'colsample_bytree' : 0.85 -> score : 11.289090541110422
- 'colsample_bytree' : 0.90 -> score : 11.193780988596988
- 'colsample_bytree' : 0.95 -> score : 11.193780988596988
- 'colsample_bytree' : 1.00 -> score : 11.27700235393081


- 'n_estimators' : 1400 -> score : 11.195739124743117
- 'n_estimators' : 1500 -> score : 11.193780988596988
- 'n_estimators' : 1600 -> score : 11.195144507584441


- 'max_depth' : 7 -> score : 11.41866426958629
- 'max_depth' : 8 -> score : 11.193780988596988
- 'max_depth' : 9 -> score : 11.173645757975832
- 'max_depth' : 10 -> score : 11.31215321717593


- 'reg_alpha' : 0 -> score : 11.173645757975832
- 'reg_alpha' : 0.5 -> score : 11.158566613863234
- 'reg_alpha' : 0.6 -> score : 11.152764218716774
- 'reg_alpha' : 0.7 -> score : 11.1442919339056
- 'reg_alpha' : 0.8 -> score : 11.163985128246354


- 'reg_lambda' : 0.9 -> score : 11.155791261430858
- 'reg_lambda' : 1 -> score : 11.1442919339056


- remove chinese new year -> score : 11.067240850915448


- param_dict = {
    'max_depth' : 9,
    'learning_rate' : 0.01,
    'n_estimators' : 1500,
    'silent' : True,
    'objective' : 'reg:linear',
    'booster' : 'gbtree',
    'n_jobs' : 8,
    'gamma' : 0,
    'min_child_weight' : 1,
    'max_delta_step' : 0,
    'subsample' : 0.75,
    'colsample_bytree' : 0.9,
    'reg_alpha' : 0.7,
    'reg_lambda' : 1 
}

In [69]:
# generate filtered train file
train1 = remove_other_hours_data(train)
train2 = remove_chinese_new_year_data(train1)
train2.reset_index()
train2.to_csv(filtered_training_features_filepath, index=False)

In [81]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

param_dict = {
    'max_depth' : 7,
    'learning_rate' : 0.01,
    'n_estimators' : 1500,
    'silent' : True,
    'objective' : 'reg:linear',
    'booster' : 'gbtree',
    'n_jobs' : 8,
    'gamma' : 0,
    'min_child_weight' : 1,
    'max_delta_step' : 0,
    'subsample' : 0.75,
    'colsample_bytree' : 0.9,
    'reg_alpha' : 0.7,
    'reg_lambda' : 1
}


# loading filtered train file
train = pd.read_csv(filtered_training_features_filepath)
# split with index, use last week's data as validation data
#index = retrieve_index_by_month_day_hour(train, 3, 6, 9)
#train_x, train_y, val_x, val_y = split_train_val_data(index)

#regressor = xgb.XGBRegressor(**param_dict)
# train with part of data
#regressor.fit(train_x.as_matrix(), train_y)
#pred = regressor.predict(val_x.as_matrix())
#print("score is ", mean_squared_error(val_y, pred))

# train with all the data
train_x = train.drop(columns=drop_columns)
train_y = train['car_number']
regressor.fit(train_x.as_matrix(), train_y)

test_ = test.drop(columns=drop_columns)
pred = regressor.predict(test_.as_matrix())

In [82]:
generate_submission_file(pred)

### Model selecting

- LGB

### parameter tuning

In [10]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'max_depth': 6,
    'learning_rate': 0.01,
    'verbose': 0}
    #'early_stopping_round': 3000}

n_estimators = 5000

index = retrieve_index_by_month_day_hour(train, 3, 6, 0)
train_x, train_y, val_x, val_y = split_train_val_data(index)

d_train = lgb.Dataset(train_x, label=train_y)
d_valid = lgb.Dataset(val_x, label=val_y)
watchlist = [d_valid]

model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

[1]	valid_0's l2: 70.5348
[2]	valid_0's l2: 69.8238
[3]	valid_0's l2: 69.1237
[4]	valid_0's l2: 68.5715
[5]	valid_0's l2: 68.0211
[6]	valid_0's l2: 67.4913
[7]	valid_0's l2: 66.8321
[8]	valid_0's l2: 66.3154
[9]	valid_0's l2: 65.8033
[10]	valid_0's l2: 65.3179
[11]	valid_0's l2: 64.8249
[12]	valid_0's l2: 64.3523
[13]	valid_0's l2: 63.7368
[14]	valid_0's l2: 63.1326
[15]	valid_0's l2: 62.6637
[16]	valid_0's l2: 62.0791
[17]	valid_0's l2: 61.6268
[18]	valid_0's l2: 61.1877
[19]	valid_0's l2: 60.7519
[20]	valid_0's l2: 60.3248
[21]	valid_0's l2: 59.9201
[22]	valid_0's l2: 59.5162
[23]	valid_0's l2: 58.976
[24]	valid_0's l2: 58.4368
[25]	valid_0's l2: 58.0533
[26]	valid_0's l2: 57.6686
[27]	valid_0's l2: 57.3125
[28]	valid_0's l2: 56.9516
[29]	valid_0's l2: 56.4723
[30]	valid_0's l2: 56.1164
[31]	valid_0's l2: 55.6407
[32]	valid_0's l2: 55.2959
[33]	valid_0's l2: 54.9109
[34]	valid_0's l2: 54.5952
[35]	valid_0's l2: 54.2657
[36]	valid_0's l2: 53.8281
[37]	valid_0's l2: 53.502
[38]	valid_0

[325]	valid_0's l2: 19.2891
[326]	valid_0's l2: 19.2499
[327]	valid_0's l2: 19.1929
[328]	valid_0's l2: 19.1791
[329]	valid_0's l2: 19.1634
[330]	valid_0's l2: 19.1563
[331]	valid_0's l2: 19.101
[332]	valid_0's l2: 19.0595
[333]	valid_0's l2: 19.0522
[334]	valid_0's l2: 19.0128
[335]	valid_0's l2: 19.0103
[336]	valid_0's l2: 18.9919
[337]	valid_0's l2: 18.9691
[338]	valid_0's l2: 18.9162
[339]	valid_0's l2: 18.9138
[340]	valid_0's l2: 18.898
[341]	valid_0's l2: 18.889
[342]	valid_0's l2: 18.8517
[343]	valid_0's l2: 18.8342
[344]	valid_0's l2: 18.8307
[345]	valid_0's l2: 18.8205
[346]	valid_0's l2: 18.7686
[347]	valid_0's l2: 18.7666
[348]	valid_0's l2: 18.7546
[349]	valid_0's l2: 18.7041
[350]	valid_0's l2: 18.701
[351]	valid_0's l2: 18.6931
[352]	valid_0's l2: 18.657
[353]	valid_0's l2: 18.6083
[354]	valid_0's l2: 18.5965
[355]	valid_0's l2: 18.5621
[356]	valid_0's l2: 18.5155
[357]	valid_0's l2: 18.4708
[358]	valid_0's l2: 18.4641
[359]	valid_0's l2: 18.4184
[360]	valid_0's l2: 18.38

[636]	valid_0's l2: 14.5878
[637]	valid_0's l2: 14.5798
[638]	valid_0's l2: 14.5694
[639]	valid_0's l2: 14.5649
[640]	valid_0's l2: 14.5606
[641]	valid_0's l2: 14.5493
[642]	valid_0's l2: 14.5398
[643]	valid_0's l2: 14.5288
[644]	valid_0's l2: 14.5189
[645]	valid_0's l2: 14.5191
[646]	valid_0's l2: 14.5083
[647]	valid_0's l2: 14.499
[648]	valid_0's l2: 14.4922
[649]	valid_0's l2: 14.4925
[650]	valid_0's l2: 14.4824
[651]	valid_0's l2: 14.4784
[652]	valid_0's l2: 14.4713
[653]	valid_0's l2: 14.4618
[654]	valid_0's l2: 14.4581
[655]	valid_0's l2: 14.4482
[656]	valid_0's l2: 14.4375
[657]	valid_0's l2: 14.4331
[658]	valid_0's l2: 14.4239
[659]	valid_0's l2: 14.4149
[660]	valid_0's l2: 14.4113
[661]	valid_0's l2: 14.4025
[662]	valid_0's l2: 14.4028
[663]	valid_0's l2: 14.399
[664]	valid_0's l2: 14.3894
[665]	valid_0's l2: 14.379
[666]	valid_0's l2: 14.3689
[667]	valid_0's l2: 14.3599
[668]	valid_0's l2: 14.356
[669]	valid_0's l2: 14.3503
[670]	valid_0's l2: 14.3507
[671]	valid_0's l2: 14.3

[970]	valid_0's l2: 12.9927
[971]	valid_0's l2: 12.9895
[972]	valid_0's l2: 12.988
[973]	valid_0's l2: 12.9834
[974]	valid_0's l2: 12.9811
[975]	valid_0's l2: 12.9826
[976]	valid_0's l2: 12.9827
[977]	valid_0's l2: 12.9812
[978]	valid_0's l2: 12.9739
[979]	valid_0's l2: 12.9693
[980]	valid_0's l2: 12.9665
[981]	valid_0's l2: 12.9637
[982]	valid_0's l2: 12.9622
[983]	valid_0's l2: 12.958
[984]	valid_0's l2: 12.956
[985]	valid_0's l2: 12.9561
[986]	valid_0's l2: 12.9536
[987]	valid_0's l2: 12.9521
[988]	valid_0's l2: 12.9499
[989]	valid_0's l2: 12.9515
[990]	valid_0's l2: 12.9496
[991]	valid_0's l2: 12.9469
[992]	valid_0's l2: 12.9454
[993]	valid_0's l2: 12.9412
[994]	valid_0's l2: 12.9386
[995]	valid_0's l2: 12.9402
[996]	valid_0's l2: 12.9373
[997]	valid_0's l2: 12.9361
[998]	valid_0's l2: 12.9361
[999]	valid_0's l2: 12.9329
[1000]	valid_0's l2: 12.9259
[1001]	valid_0's l2: 12.9217
[1002]	valid_0's l2: 12.9198
[1003]	valid_0's l2: 12.9184
[1004]	valid_0's l2: 12.919
[1005]	valid_0's l2

[1278]	valid_0's l2: 12.4314
[1279]	valid_0's l2: 12.4299
[1280]	valid_0's l2: 12.4294
[1281]	valid_0's l2: 12.4274
[1282]	valid_0's l2: 12.4269
[1283]	valid_0's l2: 12.4269
[1284]	valid_0's l2: 12.4271
[1285]	valid_0's l2: 12.4252
[1286]	valid_0's l2: 12.4259
[1287]	valid_0's l2: 12.4252
[1288]	valid_0's l2: 12.4237
[1289]	valid_0's l2: 12.4233
[1290]	valid_0's l2: 12.4214
[1291]	valid_0's l2: 12.4196
[1292]	valid_0's l2: 12.4184
[1293]	valid_0's l2: 12.4186
[1294]	valid_0's l2: 12.4185
[1295]	valid_0's l2: 12.4177
[1296]	valid_0's l2: 12.4165
[1297]	valid_0's l2: 12.4151
[1298]	valid_0's l2: 12.4141
[1299]	valid_0's l2: 12.412
[1300]	valid_0's l2: 12.4112
[1301]	valid_0's l2: 12.4095
[1302]	valid_0's l2: 12.4085
[1303]	valid_0's l2: 12.4082
[1304]	valid_0's l2: 12.4073
[1305]	valid_0's l2: 12.4062
[1306]	valid_0's l2: 12.4059
[1307]	valid_0's l2: 12.4046
[1308]	valid_0's l2: 12.4035
[1309]	valid_0's l2: 12.4031
[1310]	valid_0's l2: 12.4013
[1311]	valid_0's l2: 12.4009
[1312]	valid_0'

...

[3731]	valid_0's l2: 11.7509
[3732]	valid_0's l2: 11.7508
[3733]	valid_0's l2: 11.7511
[3734]	valid_0's l2: 11.7511
[3735]	valid_0's l2: 11.7515
[3736]	valid_0's l2: 11.7511
[3737]	valid_0's l2: 11.751
[3738]	valid_0's l2: 11.7512
[3739]	valid_0's l2: 11.7509
[3740]	valid_0's l2: 11.7508
[3741]	valid_0's l2: 11.7505
[3742]	valid_0's l2: 11.7501
[3743]	valid_0's l2: 11.7499
[3744]	valid_0's l2: 11.7496
[3745]	valid_0's l2: 11.7511
[3746]	valid_0's l2: 11.7509
[3747]	valid_0's l2: 11.7511
[3748]	valid_0's l2: 11.751
[3749]	valid_0's l2: 11.751
[3750]	valid_0's l2: 11.751
[3751]	valid_0's l2: 11.7506
[3752]	valid_0's l2: 11.7508
[3753]	valid_0's l2: 11.7506
[3754]	valid_0's l2: 11.7506
[3755]	valid_0's l2: 11.7502
[3756]	valid_0's l2: 11.7502
[3757]	valid_0's l2: 11.75
[3758]	valid_0's l2: 11.7502
[3759]	valid_0's l2: 11.7502
[3760]	valid_0's l2: 11.7503
[3761]	valid_0's l2: 11.7501
[3762]	valid_0's l2: 11.7495
[3763]	valid_0's l2: 11.7496
[3764]	valid_0's l2: 11.7497
[3765]	valid_0's l2:

[4039]	valid_0's l2: 11.7356
[4040]	valid_0's l2: 11.7353
[4041]	valid_0's l2: 11.7352
[4042]	valid_0's l2: 11.7351
[4043]	valid_0's l2: 11.7352
[4044]	valid_0's l2: 11.7351
[4045]	valid_0's l2: 11.7351
[4046]	valid_0's l2: 11.7351
[4047]	valid_0's l2: 11.7349
[4048]	valid_0's l2: 11.7349
[4049]	valid_0's l2: 11.7348
[4050]	valid_0's l2: 11.7348
[4051]	valid_0's l2: 11.7347
[4052]	valid_0's l2: 11.7348
[4053]	valid_0's l2: 11.7354
[4054]	valid_0's l2: 11.7353
[4055]	valid_0's l2: 11.7353
[4056]	valid_0's l2: 11.7353
[4057]	valid_0's l2: 11.7354
[4058]	valid_0's l2: 11.7354
[4059]	valid_0's l2: 11.735
[4060]	valid_0's l2: 11.7348
[4061]	valid_0's l2: 11.7349
[4062]	valid_0's l2: 11.7349
[4063]	valid_0's l2: 11.7345
[4064]	valid_0's l2: 11.7344
[4065]	valid_0's l2: 11.7343
[4066]	valid_0's l2: 11.7342
[4067]	valid_0's l2: 11.7342
[4068]	valid_0's l2: 11.7338
[4069]	valid_0's l2: 11.7338
[4070]	valid_0's l2: 11.7338
[4071]	valid_0's l2: 11.7339
[4072]	valid_0's l2: 11.7336
[4073]	valid_0'

[4355]	valid_0's l2: 11.6879
[4356]	valid_0's l2: 11.6878
[4357]	valid_0's l2: 11.688
[4358]	valid_0's l2: 11.688
[4359]	valid_0's l2: 11.6877
[4360]	valid_0's l2: 11.6877
[4361]	valid_0's l2: 11.6878
[4362]	valid_0's l2: 11.6876
[4363]	valid_0's l2: 11.687
[4364]	valid_0's l2: 11.687
[4365]	valid_0's l2: 11.6871
[4366]	valid_0's l2: 11.6871
[4367]	valid_0's l2: 11.687
[4368]	valid_0's l2: 11.6872
[4369]	valid_0's l2: 11.687
[4370]	valid_0's l2: 11.687
[4371]	valid_0's l2: 11.6863
[4372]	valid_0's l2: 11.6863
[4373]	valid_0's l2: 11.6862
[4374]	valid_0's l2: 11.686
[4375]	valid_0's l2: 11.6861
[4376]	valid_0's l2: 11.6862
[4377]	valid_0's l2: 11.6862
[4378]	valid_0's l2: 11.6862
[4379]	valid_0's l2: 11.6861
[4380]	valid_0's l2: 11.6862
[4381]	valid_0's l2: 11.6863
[4382]	valid_0's l2: 11.6859
[4383]	valid_0's l2: 11.6847
[4384]	valid_0's l2: 11.6848
[4385]	valid_0's l2: 11.6835
[4386]	valid_0's l2: 11.6832
[4387]	valid_0's l2: 11.6832
[4388]	valid_0's l2: 11.6829
[4389]	valid_0's l2: 1

[4643]	valid_0's l2: 11.6609
[4644]	valid_0's l2: 11.6609
[4645]	valid_0's l2: 11.6608
[4646]	valid_0's l2: 11.6608
[4647]	valid_0's l2: 11.6608
[4648]	valid_0's l2: 11.6609
[4649]	valid_0's l2: 11.6608
[4650]	valid_0's l2: 11.6603
[4651]	valid_0's l2: 11.6604
[4652]	valid_0's l2: 11.6605
[4653]	valid_0's l2: 11.6604
[4654]	valid_0's l2: 11.6605
[4655]	valid_0's l2: 11.6604
[4656]	valid_0's l2: 11.6604
[4657]	valid_0's l2: 11.6604
[4658]	valid_0's l2: 11.6603
[4659]	valid_0's l2: 11.6603
[4660]	valid_0's l2: 11.6604
[4661]	valid_0's l2: 11.6599
[4662]	valid_0's l2: 11.6601
[4663]	valid_0's l2: 11.6602
[4664]	valid_0's l2: 11.6602
[4665]	valid_0's l2: 11.6601
[4666]	valid_0's l2: 11.6603
[4667]	valid_0's l2: 11.6602
[4668]	valid_0's l2: 11.6601
[4669]	valid_0's l2: 11.6601
[4670]	valid_0's l2: 11.6602
[4671]	valid_0's l2: 11.6602
[4672]	valid_0's l2: 11.6601
[4673]	valid_0's l2: 11.6601
[4674]	valid_0's l2: 11.66
[4675]	valid_0's l2: 11.66
[4676]	valid_0's l2: 11.6601
[4677]	valid_0's l

[4953]	valid_0's l2: 11.6558
[4954]	valid_0's l2: 11.6566
[4955]	valid_0's l2: 11.6559
[4956]	valid_0's l2: 11.6556
[4957]	valid_0's l2: 11.6556
[4958]	valid_0's l2: 11.6558
[4959]	valid_0's l2: 11.6556
[4960]	valid_0's l2: 11.6557
[4961]	valid_0's l2: 11.6551
[4962]	valid_0's l2: 11.655
[4963]	valid_0's l2: 11.655
[4964]	valid_0's l2: 11.6559
[4965]	valid_0's l2: 11.6558
[4966]	valid_0's l2: 11.6557
[4967]	valid_0's l2: 11.6558
[4968]	valid_0's l2: 11.6557
[4969]	valid_0's l2: 11.6558
[4970]	valid_0's l2: 11.6554
[4971]	valid_0's l2: 11.6556
[4972]	valid_0's l2: 11.6558
[4973]	valid_0's l2: 11.6558
[4974]	valid_0's l2: 11.6557
[4975]	valid_0's l2: 11.6558
[4976]	valid_0's l2: 11.6559
[4977]	valid_0's l2: 11.656
[4978]	valid_0's l2: 11.6561
[4979]	valid_0's l2: 11.6561
[4980]	valid_0's l2: 11.6561
[4981]	valid_0's l2: 11.6561
[4982]	valid_0's l2: 11.656
[4983]	valid_0's l2: 11.656
[4984]	valid_0's l2: 11.6558
[4985]	valid_0's l2: 11.6558
[4986]	valid_0's l2: 11.6558
[4987]	valid_0's l2